# Extraction d'information à partir d'un site My GoogleMaps
Il faut extraire les couches suivantes :
 - Approvisionneurs Région NA 
 - Approvisionneurs Hors région NA
 - Approvisionneurs certifiés CBQ+
 
De la carte :
https://www.google.com/maps/d/u/0/viewer?mid=1HH5aQL1fBkZCNS0WvmbXLHktTHw&femb=1&ll=45.17435835382032%2C1.2260992670417847&z=7  \
qui provient du site suivant : https://www.crer.info/bois-energie/

In [1]:
from fastkml import kml
import pandas as pd

Lecture du fichier kml extrait à partir de la carte GoogleMaps (fichier kml)

In [2]:
kml_file = 'CRER.kml'

# Chargement du fichier KML
with open(kml_file, 'r', encoding='utf-8') as f:
    kml_content = f.read()

# Charger le contenu KML
k = kml.KML()
k.from_string(kml_content.encode('utf-8'))

No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found
No geometries found


on extrait les données dans un tableau Excel

In [3]:
# Dictionnaire pour stocker les données
data = {}

# Extraction des informations
for feature in k.features():
    for placemark in feature.features():
        appro = placemark.name 
        print(f"Approvisionneurs: {appro}")
        
        # Liste pour stocker les informations des sociétés
        societe_info = []

        for sub_feature in placemark.features():
            if isinstance(sub_feature, kml.Placemark):
                societe_name = sub_feature.name 
                info_dict = {'Société': societe_name, 'Approvisionneur': appro}

                # Si les informations sont sous forme de <Data> dans extended_data
                if hasattr(sub_feature, 'extended_data'):
                    for data_element in sub_feature.extended_data.elements:
                        data_name = data_element.name
                        data_value = data_element.value if data_element.value else "N/A"
                        info_dict[data_name] = data_value
                else:
                    societe_infos = sub_feature.description if hasattr(sub_feature, 'description') else "N/A"
                    # Sépare les informations par un ';'
                    info_fields = societe_infos.split(';')
                    
                    # Remplir le dictionnaire avec les informations séparées
                    for i, field in enumerate(info_fields):
                        info_dict[f'Info {i+1}'] = field.strip()  # Ajoute chaque champ sous une clé distincte

                societe_info.append(info_dict)

        # Ajouter les informations au dictionnaire principal
        data[appro] = societe_info

# Créer un fichier Excel avec un onglet par approvisionneur
with pd.ExcelWriter('approvisionneurs.xlsx') as writer:
    for appro, infos in data.items():
        df = pd.DataFrame(infos)
        df.to_excel(writer, sheet_name=appro[:31], index=False)  # Limiter à 31 caractères pour le nom de l'onglet

print("Les données ont été enregistrées dans 'approvisionneurs.xlsx'.")

Approvisionneurs: Approvisionneurs - Hors région Nouvelle-Aquitaine
Approvisionneurs: Limites Région NA
Approvisionneurs: Approvisionneurs Certifiées CBQ+
Approvisionneurs: Approvisionneur Région NA
Les données ont été enregistrées dans 'approvisionneurs.xlsx'.
